In [ ]:
from planet4 import io, clustering, plotting, reduction

In [ ]:
def cluster_and_plot(image_id, hdbscan_min_samples=1, scaler='robust'):
    from planet4 import plotting, clustering
    cm = clustering.ClusteringManager(do_dynamic_min_samples=True,
                                      include_angle=True,
                                      quiet=True,
                                      output_dir='new_clustering',
                                      normalize=True,
                                      scaler=scaler,
                                      min_samples_factor=0.1,
                                      use_DBSCAN=False,
                                      hdbscan_min_samples=hdbscan_min_samples)
    cm.cluster_image_id(image_id)
    plotting.plot_image_id_pipeline(image_id,
                                    save=True, 
                                    savetitle='dynamic_new',
                                    cm=cm,
                                   )
    return cm

In [ ]:
db = io.DBManager()

In [ ]:
myids = np.random.choice(ids, 100)

In [ ]:
cm = cluster_and_plot(myids[0], 3, scaler='minmax')

In [ ]:
from planet4 import markings

In [ ]:
imgid = markings.ImageID('6n3', scope='planet4')

In [ ]:
blotches = imgid.get_blotches()

In [ ]:
fans = imgid.get_fans()

In [ ]:
fans.user_name.nunique()

In [ ]:
user_names = fans.user_name.unique()

In [ ]:
%matplotlib inline

In [ ]:
from planet4.markings import set_subframe_size
from itertools import cycle
import seaborn as sns
sns.set_context('poster')
pal = sns.color_palette('colorblind', 10)
fig, ax = plt.subplots()
ax.set_title('6n3, user=input')
imgid.show_subframe(ax=ax)
fig.savefig('plots/6n3_000.png', dpi=80)
i=1
for user_name, color in zip(user_names, cycle(pal)):
    imgid.plot_fans(user_name=user_name, user_color=color, ax=ax,
                    linewidth=3)
    ax.set_title('6n3, user={}'.format(str(i).zfill(3)))
    fig.savefig('plots/6n3_{}.png'.format(str(i).zfill(3)), dpi=80)
    i+=1

In [ ]:
fans.iloc[0]

In [ ]:
data = imgid.data.assign(sinangle=np.sin(np.deg2rad(imgid.data.angle)))
data = data.assign(cosangle=np.cos(np.deg2rad(imgid.data.angle)))

In [ ]:
blotches = data[data.marking=='blotch']
fans = data[data.marking == 'fan']

In [ ]:
X = blotches[['x','y']].as_matrix()
X = blotches[['x','y', 'radius_1', 'radius_2']].as_matrix()
# X = blotches[['x','y','sinangle']].as_matrix()
X = blotches[['x','y', 'sinangle', 'radius_1', 'radius_2']].as_matrix()

In [ ]:
X = blotches[['x','y', 'cosangle', 'sinangle']].as_matrix()

In [ ]:
from sklearn.preprocessing import StandardScaler, robust_scale

scaler = StandardScaler().fit(X)

tX = robust_scale(X)

In [ ]:
import hdbscan
def do_hdbscan(min_cluster, min_samples=None):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster, 
                                approx_min_span_tree=False,
                                min_samples=min_samples,
                               )

    imgid.plot_fans()

    db = clusterer.fit(tX)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.probabilities_ > 0.75] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = 'k'

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                 markeredgecolor='k', markersize=6)

    plt.title('Estimated number of clusters: %d' % n_clusters_)
    
    reduced_data = []
    for k in unique_labels:
        class_member_mask = (labels == k)
        if k == -1:
            continue
        else:
            reduced_data.append(class_member_mask & core_samples_mask)
    return reduced_data

In [ ]:
reduced_data = do_hdbscan(4, 1)

In [ ]:
blotches[reduced_data[0]]['x y'.split()]

In [ ]:
tX.shape

In [ ]:
plt.figure()
db.condensed_tree_.plot(select_clusters=True)

In [ ]:
cm.dbname

In [ ]:
db = io.DBManager()

In [ ]:
data = db.get_image_id_markings('bvc')

In [ ]:
n_class_old = data.classification_id.nunique()
n_class_old

In [ ]:
# number of classifications that include fan and blotches
f1 = data.marking == 'fan'
f2 = data.marking == 'blotch'
n_class_fb = data[f1 | f2].classification_id.nunique()
n_class_fb

In [ ]:
data=data[data.marking=='blotch']

In [ ]:
plotting.plot_raw_blotches('bvc')

In [ ]:
data['y_R'] = 1000 - data['y']

In [ ]:
data.plot(kind='scatter', x='x',y='y_R')

In [ ]:
fx1 = data.x < 400 
fx2 = data.x > 300
fy1 = data.y_R > 300
fy2 = data.y_R < 400

In [ ]:
data = data.reset_index()

In [ ]:
data[fx1 & fx2 & fy1 & fy2].angle

In [ ]:
cm.dbscanner.reduced_data

In [ ]:
from planet4 import markings

In [ ]:
clusterdata = data[markings.Blotch.to_average].iloc[cm.dbscanner.reduced_data[1]]

In [ ]:
clusterdata

In [ ]:
meandata = clusterdata.mean()
meandata

In [ ]:
clusterdata.iloc[[0,2,3]]

In [ ]:
meandata = clusterdata.iloc[[0,2,3]].mean()
meandata

In [ ]:
from scipy.stats import circmean
meandata.angle = np.rad2deg(circmean(np.deg2rad(clusterdata.angle)))

In [ ]:
meandata

In [ ]:
np.rad2deg(circmean(np.deg2rad([1,178])))

In [ ]:
angles = np.arange(-179, 180, 5)
angles

In [ ]:
angles % 180

In [ ]:
alldata = reduction.read_csv_into_df('/Volumes/USB128GB/planet4/2016-05-29_planet_four_classifications.csv')